# Lab - Classification

In this lab, we are going to build a classification module. When given an image of a handwritten digit like the one below, the model will be able to tell which digit is in the image.

<img src='test2.jpg'>

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier  # MLP is an NN
from sklearn import svm
import numpy as np
import argparse
import imutils  # If you are unable to install this library, ask the TA; we only need this in extract_hsv_histogram.
import cv2
import os
import random


# Depending on library versions on your system, one of the following imports 
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split

In [9]:
path_to_dataset = r'digits_dataset'
target_img_size = (32, 32) # fix image size because classification algorithms THAT WE WILL USE HERE expect that

# We are going to fix the random seed to make our experiments reproducible 
# since some algorithms use pseudorandom generators
random_seed = 42  
random.seed(random_seed)
np.random.seed(random_seed)

## Part I - Feature Extraction

In this part, we are going to implement three functions. Each one will extract a different set of features from the image. The three sets are:

1. Histogram of the pixel values features (this is the histogram you know, but on the HSV channels)
2. Histogram of Gradients (HoG) features
3. Raw pixels (basically, not doing any feature extraction and just supplying the input image to the classifier)

In [10]:
def extract_hsv_histogram(img):
    """
    TODO
    1. Resize the image to target_img_size using cv2.resize #DONE
    2. Convert the image from BGR representation (cv2 is BGR not RGB) to HSV using cv2.cvtColor
    3. Acquire the histogram using the cv2.calcHist. Apply the functions on the 3 channels. For the bins 
        parameter pass (8, 8, 8). For the ranges parameter pass ([0, 180, 0, 256, 0, 256]). Name the histogram
        <hist>.
    """
    img=cv2.resize(img, target_img_size)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) #BGR to HSV vs 66 for BGR to HSV full (range of H 0-> 255 vs 0->180)

    #cv2.calcHist(images, channels, mask, histSize, ranges)
    hist = cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8], [0, 180, 0, 256, 0, 256])

    if imutils.is_cv2():
        hist = cv2.normalize(hist)
    else:
        cv2.normalize(hist, hist)
    return hist.flatten()     

In [11]:
def extract_hog_features(img):
    """
    TODO
    You won't implement anything in this function. You just need to understand it 
    and understand its parameters (i.e win_size, cell_size, ... etc)
    """
    img = cv2.resize(img, target_img_size) #resized image to (32,32)
    win_size = (32, 32)
    cell_size = (4, 4)
    block_size_in_cells = (2, 2) #we have 8n cells in image
    #divide into blocks of 2x2 cells 
    # image has 4 blocks    
    block_size = (block_size_in_cells[1] * cell_size[1], block_size_in_cells[0] * cell_size[0])
    block_stride = (cell_size[1], cell_size[0]) # why reversed? because of the way opencv is implemented
    nbins = 9  # Number of orientation bins
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
    h = hog.compute(img)
    h = h.flatten()
    return h.flatten()

In [12]:
def extract_raw_pixels(img):
    """
    TODO
    The classification algorithms we are going to use expect the input to be a vector not a matrix. 
    This is because they are general purpose and don't work only on images.
    CNNs, on the other hand, expect matrices since they operate on images and exploit the 
    arrangement of pixels in the 2-D space.
    
    So, what we only need to do in this function is to resize and flatten the image.
    """
    img = cv2.resize(img, target_img_size)
    img = img.flatten()
    return img

In [13]:
def extract_features(img, feature_set='hog'):
    """
    TODO
    Given either 'hsv_hist', 'hog', 'raw', call the respective function and return its output
    """
    switcher = {
        'hsv_hist': extract_hsv_histogram,
        'hog': extract_hog_features,
        'raw': extract_raw_pixels
    }
    return switcher[feature_set](img)

The following function will extract the features and the label of each image in our dataset and save it in RAM. We normally don't save datasets in RAM, but this dataset is small.

In [14]:
def load_dataset(feature_set='hog'):
    features = []
    labels = []
    img_filenames = os.listdir(path_to_dataset)

    for i, fn in enumerate(img_filenames):
        if fn.split('.')[-1] != 'jpg':
            continue

        label = fn.split('.')[0]
        labels.append(label)

        path = os.path.join(path_to_dataset, fn)
        img = cv2.imread(path)
        features.append(extract_features(img, feature_set))
        
        # show an update every 1,000 images
        if i > 0 and i % 1000 == 0:
            print("[INFO] processed {}/{}".format(i, len(img_filenames)))
        
    return features, labels        

## Part II - Classification

In this part, we will test the classification performance of SVM, KNN, & NNs given our features.

In [ ]:
# TODO understand the hyperparameters of each classifier
classifiers = {
    'SVM': svm.LinearSVC(random_state=random_seed),
    'KNN': KNeighborsClassifier(n_neighbors=3),
    'NN': MLPClassifier(solver='sgd', random_state=random_seed, hidden_layer_sizes=(500,), max_iter=50, verbose=1)
}

In [25]:
# This function will test all our classifiers on a specific feature set
def run_experiment(feature_set):
    
    # Load dataset with extracted features
    print('Loading dataset. This will take time ...')
    features, labels = load_dataset(feature_set)
    print('Finished loading dataset.')
    
    # Since we don't want to know the performance of our classifier on images it has seen before
    # we are going to withhold some images that we will test the classifier on after training 
    train_features, test_features, train_labels, test_labels = train_test_split(
        features, labels, test_size=0.2, random_state=random_seed)
    
    for model_name, model in classifiers.items():
        print('############## Training', model_name, "##############")
        # Train the model only on the training features
        model.fit(train_features, train_labels)
        
        # Test the model on images it hasn't seen before
        accuracy = model.score(test_features, test_labels)
        
        print(model_name, 'accuracy:', accuracy*100, '%')

Now, we see how each classifier and each feature set performs

In [ ]:
run_experiment('hog')
"""
You should get the following test accuracies the first time 

SVM accuracy ~ 97.70833333333333
KNN accuracy ~ 96.52777777777779
NN accuracy ~ 93.95833333333333
"""
#with max itr 50 
############## Training SVM ##############
#SVM accuracy: 97.43055555555556 %
############## Training KNN ##############
#KNN accuracy: 96.80555555555556 %
############## Training NN ##############
#NN accuracy: 95.34722222222223 %


Loading dataset. This will take time ...
[INFO] processed 1000/7200
[INFO] processed 2000/7200
[INFO] processed 3000/7200
[INFO] processed 4000/7200
[INFO] processed 5000/7200
[INFO] processed 6000/7200
[INFO] processed 7000/7200
Finished loading dataset.
############## Training SVM ##############
SVM accuracy: 97.43055555555556 %
############## Training KNN ##############
KNN accuracy: 96.80555555555556 %
############## Training NN ##############
Iteration 1, loss = 2.15856157
Iteration 2, loss = 1.99729607
Iteration 3, loss = 1.83723722
Iteration 4, loss = 1.68403547
Iteration 5, loss = 1.53617217
Iteration 6, loss = 1.39650686
Iteration 7, loss = 1.26711559
Iteration 8, loss = 1.15002170
Iteration 9, loss = 1.04579189
Iteration 10, loss = 0.95388062
Iteration 11, loss = 0.87339218
Iteration 12, loss = 0.80325679
Iteration 13, loss = 0.74236312
Iteration 14, loss = 0.68890729
Iteration 15, loss = 0.64224823
Iteration 16, loss = 0.60140057
Iteration 17, loss = 0.56539334
Iteration 18,

/home/sarah/anaconda3/envs/img_processing/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


'\nYou should get the following test accuracies the first time \n\nSVM accuracy ~ 97.70833333333333\nKNN accuracy ~ 96.52777777777779\nNN accuracy ~ 93.95833333333333\n'

In [ ]:
run_experiment('hsv_hist')
"""
You should get the following test accuracies the first time 

SVM accuracy ~ 32.083333333333336
KNN accuracy ~ 32.708333333333336
NN accuracy ~ 9.722222222222223
"""
#with max itr 50 


# Why low accuracies? because the features extracted from the HSV are just colors and don't have a shape 

Loading dataset. This will take time ...
[INFO] processed 1000/7200
[INFO] processed 2000/7200
[INFO] processed 3000/7200
[INFO] processed 4000/7200
[INFO] processed 5000/7200
[INFO] processed 6000/7200
[INFO] processed 7000/7200
Finished loading dataset.
############## Training SVM ##############
SVM accuracy: 32.98611111111111 %
############## Training KNN ##############
KNN accuracy: 31.38888888888889 %
############## Training NN ##############
Iteration 1, loss = 2.20272909
Iteration 2, loss = 2.20150481
Iteration 3, loss = 2.20050231
Iteration 4, loss = 2.19970445
Iteration 5, loss = 2.19902864
Iteration 6, loss = 2.19850715
Iteration 7, loss = 2.19810450
Iteration 8, loss = 2.19780968
Iteration 9, loss = 2.19748080
Iteration 10, loss = 2.19729002
Iteration 11, loss = 2.19709352
Iteration 12, loss = 2.19693706
Iteration 13, loss = 2.19680808
Iteration 14, loss = 2.19671440
Iteration 15, loss = 2.19664537
Iteration 16, loss = 2.19658561
Iteration 17, loss = 2.19651775
Iteration 18,

'\nYou should get the following test accuracies the first time \n\nSVM accuracy ~ 32.083333333333336\nKNN accuracy ~ 32.708333333333336\nNN accuracy ~ 9.722222222222223\n'

In [ ]:
run_experiment('raw')
"""
You should get the following test accuracies the first time 

SVM accuracy ~ 85.06944444444444
KNN accuracy ~ 93.95833333333333
NN accuracy ~ 88.68055555555556
"""
# with 50 itr 
#SVM accuracy: 82.91666666666667 %
############## Training KNN ##############
#KNN accuracy: 94.375 %
############## Training NN ##############
# NN accuracy: 89.72222222222223 %

Loading dataset. This will take time ...
[INFO] processed 1000/7200
[INFO] processed 2000/7200
[INFO] processed 3000/7200
[INFO] processed 4000/7200
[INFO] processed 5000/7200
[INFO] processed 6000/7200
[INFO] processed 7000/7200
Finished loading dataset.
############## Training SVM ##############


/home/sarah/anaconda3/envs/img_processing/lib/python3.9/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


SVM accuracy: 82.91666666666667 %
############## Training KNN ##############
KNN accuracy: 94.375 %
############## Training NN ##############
Iteration 1, loss = 9.72686764
Iteration 2, loss = 1.14668077
Iteration 3, loss = 0.94446195
Iteration 4, loss = 0.73025792
Iteration 5, loss = 0.61427354
Iteration 6, loss = 0.51020100
Iteration 7, loss = 0.46347263
Iteration 8, loss = 0.45281525
Iteration 9, loss = 0.44193702
Iteration 10, loss = 0.37593514
Iteration 11, loss = 0.36592295
Iteration 12, loss = 0.32526623
Iteration 13, loss = 0.29667244
Iteration 14, loss = 0.28674704
Iteration 15, loss = 0.28680128
Iteration 16, loss = 0.25673271
Iteration 17, loss = 0.24361837
Iteration 18, loss = 0.24766172
Iteration 19, loss = 0.24157142
Iteration 20, loss = 0.21627647
Iteration 21, loss = 0.19947636
Iteration 22, loss = 0.18681498
Iteration 23, loss = 0.20328489
Iteration 24, loss = 0.26911024
Iteration 25, loss = 0.21789060
Iteration 26, loss = 0.20948026
Iteration 27, loss = 0.20225637
Ite

/home/sarah/anaconda3/envs/img_processing/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


'\nYou should get the following test accuracies the first time \n\nSVM accuracy ~ 85.06944444444444\nKNN accuracy ~ 93.95833333333333\nNN accuracy ~ 88.68055555555556\n'

The classifiers list now has models trained on the last feature set you ran an experiment on. You can play around with it checking the probability it gives to each label, given an image.

In [29]:
# Example
test_img_path = 'test2.jpg'
img = cv2.imread(test_img_path)
features = extract_features(img, 'raw')  # be careful of the choice of feature set

In [30]:
nn = classifiers['NN']
nn.predict_proba([features])

array([[2.41324100e-21, 9.99999643e-01, 1.48425668e-09, 6.07924662e-18,
        8.57895916e-09, 1.69214819e-10, 3.46612999e-07, 1.27440132e-47,
        5.35384213e-23]])

Try to get a better accuracy by changing the model hyperparameters and retraining.